In [4]:
import pandas as pd

# Load the dataset
#Replace with your actual file path
permits_df = pd.read_csv("/Users/surajtk/Documents/ITMD-526/Data Visualization - Dashboards and Reports/Data visualization/Datasets/building-permits.csv", low_memory=False)

# Sample view
permits_df.head()

,ID,PERMIT#,PERMIT_TYPE,REVIEW_TYPE,APPLICATION_START_DATE,ISSUE_DATE,PROCESSING_TIME,STREET_NUMBER,STREET DIRECTION,STREET_NAME,...,YCOORDINATE,LATITUDE,LONGITUDE,LOCATION,Boundaries - ZIP Codes,Community Areas,Zip Codes,Census Tracts,Wards,:@computed_region_awaf_s7ux
0,2530006,100523607,PERMIT - SIGNS,SIGN PERMIT,2014-02-24T00:00:00.000,2014-03-04T00:00:00.000,8.0,10000,W,OHARE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2506958,100510546,PERMIT - SIGNS,SIGN PERMIT,2013-11-18T00:00:00.000,2013-12-19T00:00:00.000,31.0,10000,N,BESSIE COLEMAN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2506971,100510558,PERMIT - SIGNS,SIGN PERMIT,2013-11-18T00:00:00.000,2013-12-19T00:00:00.000,31.0,10000,N,BESSIE COLEMAN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2818648,100687794,PERMIT - ELECTRIC WIRING,EASY PERMIT WEB,2017-01-25T00:00:00.000,2017-01-25T00:00:00.000,0.0,150,N,RIVERSIDE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1779434,100163126,PERMIT - ELECTRIC WIRING,EASY PERMIT WEB,2007-02-15T00:00:00.000,2007-02-15T00:00:00.000,0.0,6431,S,COTTAGE GROVE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Clean column names
permits_df.columns = (
    permits_df.columns
    .str.strip()
    .str.lower()
    .str.replace(' ', '_')
    .str.replace('#', 'number')
)
print(permits_df.columns.tolist())


['id', 'permitnumber', 'permit_type', 'review_type', 'application_start_date', 'issue_date', 'processing_time', 'street_number', 'street_direction', 'street_name', 'suffix', 'work_description', 'building_fee_paid', 'zoning_fee_paid', 'other_fee_paid', 'subtotal_paid', 'building_fee_unpaid', 'zoning_fee_unpaid', 'other_fee_unpaid', 'subtotal_unpaid', 'building_fee_waived', 'zoning_fee_waived', 'other_fee_waived', 'subtotal_waived', 'total_fee', 'contact_1_type', 'contact_1_name', 'contact_1_city', 'contact_1_state', 'contact_1_zipcode', 'contact_2_type', 'contact_2_name', 'contact_2_city', 'contact_2_state', 'contact_2_zipcode', 'contact_3_type', 'contact_3_name', 'contact_3_city', 'contact_3_state', 'contact_3_zipcode', 'contact_4_type', 'contact_4_name', 'contact_4_city', 'contact_4_state', 'contact_4_zipcode', 'contact_5_type', 'contact_5_name', 'contact_5_city', 'contact_5_state', 'contact_5_zipcode', 'contact_6_type', 'contact_6_name', 'contact_6_city', 'contact_6_state', 'contact_

In [6]:

# Drop rows with missing lat/lon or ZIP
permits_df.dropna(subset=['latitude', 'longitude', 'zip_codes'], inplace=True)


In [7]:

# Select only relevant columns (now all lowercase with underscores)
columns_to_keep = [
    'id', 'permitnumber', 'permit_type', 'review_type',
    'issue_date', 'processing_time', 'street_number', 'street_direction', 'street_name',
    'zip_codes', 'reported_cost', 'latitude', 'longitude', 'community_area'
]
columns_to_keep = [col for col in columns_to_keep if col in permits_df.columns]
permits_df = permits_df[columns_to_keep]

# Convert issue_date to datetime
permits_df['issue_date'] = pd.to_datetime(permits_df['issue_date'], errors='coerce')

# Filter to permits from 2018 onwards
permits_df = permits_df[permits_df['issue_date'] >= '2018-01-01']

# Reset index
permits_df.reset_index(drop=True, inplace=True)

# Export cleaned permits data
permits_df.to_csv("building_permits_cleaned.csv", index=False)
print("Exported building_permits_cleaned.csv")

# Preview cleaned permits
print(permits_df.head())

# Summarize permit activity by community area
grouped = permits_df.groupby('community_area').agg(
    total_permits=('permit_type', 'count'),
    total_reported_cost=('reported_cost', 'sum')
).reset_index()

# Export summary
grouped.to_csv("building_permits_summary.csv", index=False)
print("Exported building_permits_summary.csv")

Exported building_permits_cleaned.csv
        id permitnumber                     permit_type           review_type  \
0  3004830    100801182                  PERMIT - SIGNS           SIGN PERMIT   
1  3052220    100830091  PERMIT - RENOVATION/ALTERATION  STANDARD PLAN REVIEW   
2  2883323    100727449  PERMIT - RENOVATION/ALTERATION  STANDARD PLAN REVIEW   
3  3075537    100844208    PERMIT - EASY PERMIT PROCESS           EASY PERMIT   
4  2919700    100748558  PERMIT - RENOVATION/ALTERATION  STANDARD PLAN REVIEW   

  issue_date  processing_time  street_number street_direction street_name  \
0 2019-07-15            108.0            231                S    LA SALLE   
1 2019-07-13              0.0           6419                N   FAIRFIELD   
2 2018-01-23             54.0            843                W    ARMITAGE   
3 2019-10-03              0.0           4236                N  WINCHESTER   
4 2018-11-06            189.0           3122                S     CALUMET   

   zip_codes

In [8]:
import pandas as pd

# Load the CSV
permits_df = pd.read_csv("/Users/surajtk/Downloads/archive/building-permits.csv", low_memory=False)

# Strip and normalize column names
permits_df.columns = permits_df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('#', 'number')

# Print column names to confirm
print(permits_df.columns.tolist())


['id', 'permitnumber', 'permit_type', 'review_type', 'application_start_date', 'issue_date', 'processing_time', 'street_number', 'street_direction', 'street_name', 'suffix', 'work_description', 'building_fee_paid', 'zoning_fee_paid', 'other_fee_paid', 'subtotal_paid', 'building_fee_unpaid', 'zoning_fee_unpaid', 'other_fee_unpaid', 'subtotal_unpaid', 'building_fee_waived', 'zoning_fee_waived', 'other_fee_waived', 'subtotal_waived', 'total_fee', 'contact_1_type', 'contact_1_name', 'contact_1_city', 'contact_1_state', 'contact_1_zipcode', 'contact_2_type', 'contact_2_name', 'contact_2_city', 'contact_2_state', 'contact_2_zipcode', 'contact_3_type', 'contact_3_name', 'contact_3_city', 'contact_3_state', 'contact_3_zipcode', 'contact_4_type', 'contact_4_name', 'contact_4_city', 'contact_4_state', 'contact_4_zipcode', 'contact_5_type', 'contact_5_name', 'contact_5_city', 'contact_5_state', 'contact_5_zipcode', 'contact_6_type', 'contact_6_name', 'contact_6_city', 'contact_6_state', 'contact_

In [9]:
import pandas as pd

# Load the CSV
permits_df = pd.read_csv("/Users/surajtk/Downloads/archive/building-permits.csv", low_memory=False)

# Normalize column names
permits_df.columns = permits_df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('#', 'number')

# Final list of useful columns to keep
columns_to_keep = [
    'id', 'permitnumber', 'permit_type', 'review_type',
    'issue_date', 'processing_time',
    'street_number', 'street_direction', 'street_name', 'suffix',
    'work_description', 'reported_cost',
    'latitude', 'longitude', 'community_area', 'zip_codes'
]

# Keep only the relevant columns (skip missing ones gracefully)
columns_to_keep = [col for col in columns_to_keep if col in permits_df.columns]
permits_df = permits_df[columns_to_keep]

# Convert issue_date to datetime
permits_df['issue_date'] = pd.to_datetime(permits_df['issue_date'], errors='coerce')

# Filter for permits issued from 2018 onward
permits_df = permits_df[permits_df['issue_date'] >= '2018-01-01']

# Fill missing costs with 0 (if needed)
permits_df['reported_cost'] = permits_df['reported_cost'].fillna(0)

# Drop rows missing geo or community info
permits_df.dropna(subset=['latitude', 'longitude', 'community_area'], inplace=True)

# Reset index for consistency
permits_df.reset_index(drop=True, inplace=True)

# Export cleaned permits data
permits_df.to_csv("building_permits_cleaned.csv", index=False)
print(("Saved the data to building_permits_cleaned.csv"))

# Group and summarize by community area
permits_summary = permits_df.groupby('community_area').agg(
    total_permits=('permit_type', 'count'),
    total_reported_cost=('reported_cost', 'sum')
).reset_index()

# Summarize permit activity by community area
grouped = permits_df.groupby('community_area').agg(
    total_permits=('permit_type', 'count'),
    total_reported_cost=('reported_cost', 'sum')
).reset_index()

# Export summary
grouped.to_csv("building_permits_summary.csv", index=False)
print(("Saved the data to building_permits_summary.csv"))

# Final preview
print("Cleaned Permits DataFrame:")
print(permits_df.head())
print(f"\nShape: {permits_df.shape}")
print("Summary of permit activity")
print(grouped.head())


Saved the data to building_permits_cleaned.csv
Saved the data to building_permits_summary.csv
Cleaned Permits DataFrame:
        id permitnumber                     permit_type           review_type  \
0  3004830    100801182                  PERMIT - SIGNS           SIGN PERMIT   
1  3052220    100830091  PERMIT - RENOVATION/ALTERATION  STANDARD PLAN REVIEW   
2  2883323    100727449  PERMIT - RENOVATION/ALTERATION  STANDARD PLAN REVIEW   
3  3075537    100844208    PERMIT - EASY PERMIT PROCESS           EASY PERMIT   
4  2919700    100748558  PERMIT - RENOVATION/ALTERATION  STANDARD PLAN REVIEW   

  issue_date  processing_time  street_number street_direction street_name  \
0 2019-07-15            108.0            231                S    LA SALLE   
1 2019-07-13              0.0           6419                N   FAIRFIELD   
2 2018-01-23             54.0            843                W    ARMITAGE   
3 2019-10-03              0.0           4236                N  WINCHESTER   
4 2018-